## Dates



In [1]:
using Dates

Date(2022, 1, 1)

2022-01-01

In [2]:
Date(Dates.Year(1974),Dates.Month(10),Dates.Day(20))

1974-10-20

In [3]:
Date("1983-07-03", dateformat"y-m-d")

1983-07-03

In [6]:
Date("1983-07-03", dateformat"yyyymmdd")

LoadError: ArgumentError: Unable to parse date time. Expected directive DatePart(mm) at char 5

In [7]:
df = DateFormat("y-m-d")
dt = Date("2015-01-01",df)

2015-01-01

In [8]:
years = ["2015", "2016"]
Date.(years, DateFormat("yyyy"))

2-element Vector{Date}:
 2015-01-01
 2016-01-01

In [14]:
dt = Date(2012,2,29)
dt2 = Date(2000,2,1)

println(dt > dt2)

println(dt != dt2)

#println(dt + dt2)
#println(dt * dt2)
#println(dt / dt2)

println(dt - dt2)

println(dt2 - dt)

true
true
4411 days
-4411 days


In [16]:
t = Date(2014, 1, 31)
println(Dates.year(t))
println(Dates.month(t))
println(Dates.week(t))
println(Dates.day(t))

println(Dates.yearmonth(t))
println(Dates.monthday(t))
println(Dates.yearmonthday(t))

2014
1
5
31
(2014, 1)
(1, 31)
(2014, 1, 31)


In [5]:
t = Date(2014, 1, 31)

println(Dates.dayofweek(t))
println(Dates.dayname(t))
println(Dates.dayofweekofmonth(t)) # 5th Friday of January
println(Dates.monthname(t))
println(Dates.daysinmonth(t))

println(Dates.isleapyear(t))
println(Dates.dayofyear(t))
println(Dates.quarterofyear(t))
println(Dates.dayofquarter(t))

# The dayname and monthname methods can also take an optional locale keyword that can be used to return the name of the day or month of the year for other languages/locales. There are also versions of these functions returning the abbreviated names, namely dayabbr and monthabbr. First the mapping is loaded into the LOCALES variable:

french_months = ["janvier", "février", "mars", "avril", "mai", "juin",
                        "juillet", "août", "septembre", "octobre", "novembre", "décembre"]

french_monts_abbrev = ["janv","févr","mars","avril","mai","juin",
                              "juil","août","sept","oct","nov","déc"]

french_days = ["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"]

Dates.LOCALES["french"] = Dates.DateLocale(french_months, french_monts_abbrev, french_days, [""])
#The above mentioned functions can then be used to perform the queries:

println(Dates.dayname(t;locale="french"))
println(Dates.monthname(t;locale="french"))
println(Dates.monthabbr(t;locale="french"))

#Since the abbreviated versions of the days are not loaded, trying to use the function dayabbr will error.

println(Dates.dayabbr(t;locale="french"))

5
Friday
5
January
31
false
31
1
31
vendredi
janvier
janv


LoadError: BoundsError: attempt to access 1-element Vector{String} at index [5]

## TimeType-Period Arithmetic

The Dates module approach tries to follow the simple principle of trying to change as little as possible when doing Period arithmetic. This approach is also often known as calendrical arithmetic. Why all the fuss about this? Let's take a classic example: add 1 month to January 31st, 2014. What's the answer? Javascript will say March 3 (assumes 31 days). PHP says March 2 (assumes 30 days). The fact is, there is no right answer. In the Dates module, it gives the result of February 28th. How does it figure that out? Consider the classic 7-7-7 gambling game in casinos.

Now just imagine that instead of 7-7-7, the slots are Year-Month-Day, or in our example, 2014-01-31. When you ask to add 1 month to this date, the month slot is incremented, so now we have 2014-02-31. Then the day number is checked if it is greater than the last valid day of the new month; if it is (as in the case above), the day number is adjusted down to the last valid day (28). What are the ramifications with this approach? Go ahead and add another month to our date, 2014-02-28 + Month(1) == 2014-03-28. What? Were you expecting the last day of March? Nope, sorry, remember the 7-7-7 slots. As few slots as possible are going to change, so we first increment the month slot by 1, 2014-03-28, and boom, we're done because that's a valid date. On the other hand, if we were to add 2 months to our original date, 2014-01-31, then we end up with 2014-03-31, as expected. The other ramification of this approach is a loss in associativity when a specific ordering is forced (i.e. adding things in different orders results in different outcomes).

In [6]:
println((Date(2014,1,29)+Dates.Day(1)) + Dates.Month(1))
println((Date(2014,1,29)+Dates.Month(1)) + Dates.Day(1))

2014-02-28
2014-03-01


What's going on there? In the first line, we're adding 1 day to January 29th, which results in 2014-01-30; then we add 1 month, so we get 2014-02-30, which then adjusts down to 2014-02-28. In the second example, we add 1 month first, where we get 2014-02-29, which adjusts down to 2014-02-28, and then add 1 day, which results in 2014-03-01. One design principle that helps in this case is that, in the presence of multiple Periods, the operations will be ordered by the Periods' types, not their value or positional order; this means Year will always be added first, then Month, then Week, etc. Hence the following does result in associativity and Just Works:

In [8]:
println(Date(2014,1,29) + Dates.Day(1) + Dates.Month(1))
println(Date(2014,1,29) + Dates.Month(1) + Dates.Day(1))

2014-03-01
2014-03-01


The bottom line is to be aware that explicitly forcing a certain associativity, when dealing with months, may lead to some unexpected results, but otherwise, everything should work as expected. 

As a bonus, all period arithmetic objects work directly with ranges:


In [10]:
dr = Date(2014,1,29):Day(1):Date(2014,2,3)
collect(dr)

6-element Vector{Date}:
 2014-01-29
 2014-01-30
 2014-01-31
 2014-02-01
 2014-02-02
 2014-02-03

In [11]:
dr = Date(2014,1,29):Dates.Month(1):Date(2014,07,29)
collect(dr)

7-element Vector{Date}:
 2014-01-29
 2014-02-28
 2014-03-29
 2014-04-29
 2014-05-29
 2014-06-29
 2014-07-29

## Adjuster Functions

As convenient as date-period arithmetic is, often the kinds of calculations needed on dates take on a calendrical or temporal nature rather than a fixed number of periods. Holidays are a perfect example; most follow rules such as "Memorial Day = Last Monday of May", or "Thanksgiving = 4th Thursday of November". These kinds of temporal expressions deal with rules relative to the calendar, like first or last of the month, next Tuesday, or the first and third Wednesdays, etc.

The Dates module provides the adjuster API through several convenient methods that aid in simply and succinctly expressing temporal rules. The first group of adjuster methods deal with the first and last of weeks, months, quarters, and years. They each take a single TimeType as input and return or adjust to the first or last of the desired period relative to the input.

In [12]:
println(Dates.firstdayofweek(Date(2014,7,16))) # Adjusts the input to the Monday of the input's week
println(Dates.lastdayofmonth(Date(2014,7,16))) # Adjusts to the last day of the input's month
println(Dates.lastdayofquarter(Date(2014,7,16))) # Adjusts to the last day of the input's quarter

2014-07-14
2014-07-31
2014-09-30


The next two higher-order methods, tonext, and toprev, generalize working with temporal expressions by taking a DateFunction as first argument, along with a starting TimeType. A DateFunction is just a function, usually anonymous, that takes a single TimeType as input and returns a Bool, true indicating a satisfied adjustment criterion. For example:

In [14]:
istuesday = x->Dates.dayofweek(x) == Dates.Tuesday # Returns true if the day of the week of x is Tuesday
println(Dates.tonext(istuesday, Date(2014,7,13))) # 2014-07-13 is a Sunday
println(Dates.tonext(Date(2014,7,13), Dates.Tuesday)) # Convenience method provided for day of the week adjustments

2014-07-15
2014-07-15


This is useful with the do-block syntax for more complex temporal expressions:

In [15]:
Dates.tonext(Date(2014,7,13)) do x
   # Return true on the 4th Thursday of November (Thanksgiving)
   Dates.dayofweek(x) == Dates.Thursday &&
   Dates.dayofweekofmonth(x) == 4 &&
   Dates.month(x) == Dates.November
end

2014-11-27

The Base.filter method can be used to obtain all valid dates/moments in a specified range:

In [16]:
# Pittsburgh street cleaning; Every 2nd Tuesday from April to November
# Date range from January 1st, 2014 to January 1st, 2015

dr = Dates.Date(2014):Day(1):Dates.Date(2015)
filter(dr) do x
   Dates.dayofweek(x) == Dates.Tue &&
   Dates.April <= Dates.month(x) <= Dates.Nov &&
   Dates.dayofweekofmonth(x) == 2
end

8-element Vector{Date}:
 2014-04-08
 2014-05-13
 2014-06-10
 2014-07-08
 2014-08-12
 2014-09-09
 2014-10-14
 2014-11-11